In [1]:
import requests
import numpy as np
import pandas as pd
import re
import datetime
import os

#Основные настройки

In [2]:
DatasetDir=os.path.join(os.getcwd(),'/content/dataset/')

#Загрузка и распаковка исходного датасета

In [3]:
#сылка на файл: https://drive.google.com/file/d/13ojWvYkBtaiDEg1134nn6-hl9AjWCpFA/view?usp=drive_link
DataSetURL = "https://drive.google.com/uc?export=download&id=13ojWvYkBtaiDEg1134nn6-hl9AjWCpFA"

r = requests.get(DataSetURL)
with open("dataset.zip",'wb') as f:
  f.write(r.content)

##Распаковка

In [4]:
!mkdir /content/dataset
!unzip /content/dataset.zip -d $DatasetDir

Archive:  /content/dataset.zip
  inflating: /content/dataset/ЦАТ_BUZD.xlsx  
  inflating: /content/dataset/ЦАТ_GKB5.xlsx  
  inflating: /content/dataset/ЦАТ_KMRT.xlsx  
  inflating: /content/dataset/ЦАТ_NEFT.xlsx  
  inflating: /content/dataset/ЦАТ_OKT.xlsx  
  inflating: /content/dataset/ЦАТ_SIB.xlsx  
  inflating: /content/dataset/ЦАТ_STR.xlsx  
  inflating: /content/dataset/Чишминская ЦРБ.xlsx  


#Вспомогательные функции

##Работа с датой

In [5]:
def get_date(arg):
  '''
  выделяет  дату из строки
  '''
  #Ручное Имправление некоторых косяков
  arg=arg.replace('02.023.1949','02.02.1949').replace('29.062022','29.06.2022').replace('14.112022','14.11.2022').replace('00:00:00','').replace('31.11.2022','30.11.2022')
  arg=arg.replace('24.11.2022 Г ., 16.01.2023Г','16.01.2023Г')
  arg=arg.replace('-','.').replace(',','.').replace('..','.')

  #дата указана в формате 01.01.2022
  pattern=r'[\d]{1,2}[.,-][\d]{1,2}[.,-][\d]{4}'
  result = re.findall(pattern, arg)
  if len(result)==1:
    return result[0]
  #Возможно дата указана в формате 2022.01.01
  pattern=r'[\d]{4}[.,-][\d]{1,2}[.,-][\d]{1,2}'
  result = re.findall(pattern, arg)
  if len(result)==1:
    result=result[0].split('.')
    inverse_result=f'{result[2]}.{result[1]}.{result[0]}' #перевернем
    return inverse_result
 #Возможно дата указана в другом формате 08.12.22
  pattern=r'[\d]{1,2}[.,-][\d]{1,2}[.,-][\d]{2}'
  result = re.findall(pattern, arg)
  if len(result)==1:
    result=result[0].split('.')
    inverse_result=f'{result[0]}.{result[1]}.20{result[0]}'
    return inverse_result
  return ''


In [6]:
def parse_datetime (arg):
  arg = arg.replace(',','.').replace('-','.')
  arg = get_date(arg)
  try:
    d= datetime.datetime.strptime(arg, '%d.%m.%Y')
  except ValueError as err:
    return None
  return d


In [7]:
def get_valid_date(arg,min_year=None):
  '''
  Парсит дату, Проверяет правильность даты, возвращает объект datetime, если дата не валидна, вернет None
  '''
  dt=parse_datetime(arg)
  if dt is None:
    return None
  if min_year is not None and dt.year <min_year:
    return None
  return dt




In [8]:
def get_age_num(age):
  '''
  Получает на вход данные из столбцов 'Дата рождения ', пытется получить возраст (если указан именно возраст)
  В случае успех вернет число >0, иначе 0
  '''
  age_res=0
  try:
    age_res=int(age)
  except ValueError as err:
    return 0
  return age_res


In [9]:
def clear_date(arg):
  arg=str(arg).upper ().replace('ЛЕТ','').replace('ГОДА','').replace('ГОД','').replace('Г.','').strip()
  return arg

In [10]:
def get_age(b_date, a_date):
  '''
  Получает на вход данные из столбцов 'Дата рождения ' и 'Дата последнего анализа'
  Вычисляет возраст пациента на момент анализа.
  b_date - дата рождения
  a_date - дата анализа
  '''
  b_date=clear_date(b_date)
  age=get_age_num(b_date)
  if age>0:
    return age # Получили возраст
  #Получить возраст не удалось, пробуем парсить поля как дату
  b_date_parsed=get_valid_date(b_date)
  if b_date_parsed is None:
    print (f'Ошибка преобразования даты рождения: {b_date}')
    return 0
  a_date=clear_date(a_date)
  a_date_parsed=get_valid_date(a_date)
  if a_date_parsed is None:
    print (f'Ошибка преобразования даты анализа: {a_date}')
    return 0
  duration=a_date_parsed-b_date_parsed
  return duration.days//365


In [11]:
def get_age_for_row (row):
  return get_age(row['Дата рождения '],row['Дата последнего анализа'])

##Формируем общий датасет из исходных файлов

In [12]:
#Посмотрим, какие есть файлы
file_names=os.listdir(DatasetDir)
file_list =[os.path.join(DatasetDir,name) for name in file_names]
file_list

['/content/dataset/ЦАТ_GKB5.xlsx',
 '/content/dataset/ЦАТ_SIB.xlsx',
 '/content/dataset/ЦАТ_STR.xlsx',
 '/content/dataset/ЦАТ_OKT.xlsx',
 '/content/dataset/ЦАТ_NEFT.xlsx',
 '/content/dataset/ЦАТ_KMRT.xlsx',
 '/content/dataset/Чишминская ЦРБ.xlsx',
 '/content/dataset/ЦАТ_BUZD.xlsx']

In [13]:
#формируем общий датафрейм
summary_df=pd.DataFrame()
for i, file_name in enumerate(file_list):
  df=pd.read_excel(file_name)
  summary_df=pd.concat([summary_df,df])

In [14]:
summary_df

,№,МО прикрепления,Ф.И.О. пациента,Пол,Дата рождения,Диагноз,Сердечно -сосудистые события,Наименование препарата,Доза,Дата последнего анализа,Последнее значение,Показатель ПТИ,Unnamed: 12
0,1.0,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-26 00:00:00,2.4,40,NaN
1,2.0,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-13 00:00:00,2.33,27,NaN
2,3.0,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-12-05 00:00:00,1.98,NaN,NaN
3,4.0,ГБУЗ РБ ГКБ №5,Егорова Антонида Александровна,женский,1952-02-06 00:00:00,Фибрилляция предсердий,NaN,Варфарин,1 табл.,2023-02-01 00:00:00,1.83,56,NaN
4,5.0,ГБУЗ РБ ГКБ №5,Егорова Антонида Александровна,женский,1952-02-06 00:00:00,Фибрилляция предсердий,NaN,Варфарин,1 табл.,2022-12-20 00:00:00,2.53,25,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-12 00:00:00,1.66,NaN,NaN
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-12 00:00:00,1.28,0.698,NaN
96,34.0,ГБУЗ РБ Буздякская ЦРБ,Амирова Елена Хатмиевна,жен,1952-04-25 00:00:00,ИБС.Персиситирующая форма фибрилляции предсердий,NaN,Варфарин,"1,75 таб",2022-12-29 00:00:00,2.99,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-23 00:00:00,1.13,0.778,NaN


In [15]:
summary_df.dtypes

№                               float64
МО прикрепления                  object
Ф.И.О. пациента                  object
Пол                              object
Дата рождения                    object
Диагноз                          object
Сердечно -сосудистые события     object
Наименование препарата           object
Доза                             object
Дата последнего анализа          object
Последнее значение               object
Показатель ПТИ                   object
Unnamed: 12                      object
dtype: object

In [16]:
#Удаляем строки, в которых нет данных о дате анализа и результата анализа
summary_df=summary_df[~summary_df['Последнее значение'].isna() & ~summary_df['Дата последнего анализа'].isna()]

In [17]:
# удаляем лишние столбцы
summary_df=summary_df.drop(labels=['Показатель ПТИ','Unnamed: 12','№'],axis=1)
summary_df

,МО прикрепления,Ф.И.О. пациента,Пол,Дата рождения,Диагноз,Сердечно -сосудистые события,Наименование препарата,Доза,Дата последнего анализа,Последнее значение
0,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-26 00:00:00,2.4
1,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-13 00:00:00,2.33
2,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-12-05 00:00:00,1.98
3,ГБУЗ РБ ГКБ №5,Егорова Антонида Александровна,женский,1952-02-06 00:00:00,Фибрилляция предсердий,NaN,Варфарин,1 табл.,2023-02-01 00:00:00,1.83
4,ГБУЗ РБ ГКБ №5,Егорова Антонида Александровна,женский,1952-02-06 00:00:00,Фибрилляция предсердий,NaN,Варфарин,1 табл.,2022-12-20 00:00:00,2.53
...,...,...,...,...,...,...,...,...,...,...
94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-12 00:00:00,1.66
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-12 00:00:00,1.28
96,ГБУЗ РБ Буздякская ЦРБ,Амирова Елена Хатмиевна,жен,1952-04-25 00:00:00,ИБС.Персиситирующая форма фибрилляции предсердий,NaN,Варфарин,"1,75 таб",2022-12-29 00:00:00,2.99
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-23 00:00:00,1.13


In [18]:
summary_df.columns

Index(['МО прикрепления', 'Ф.И.О. пациента', 'Пол', 'Дата рождения ',
       'Диагноз', 'Сердечно -сосудистые события', 'Наименование препарата',
       'Доза', 'Дата последнего анализа', 'Последнее значение'],
      dtype='object')

##Заполняем пустые ячейки в таблице

In [19]:
#формируем индексы для столбцов, которые должны быть заполнены на основе предыдущих значений
columns=['МО прикрепления','Ф.И.О. пациента','Пол','Дата рождения ','Диагноз','Сердечно -сосудистые события','Наименование препарата','Доза']
columns_index=[list(summary_df.columns).index(x) for x in columns]
print(f'индексы заполняемых  столбцов: {columns_index}')

индексы заполняемых  столбцов: [0, 1, 2, 3, 4, 5, 6, 7]


In [20]:
num_rows=summary_df.shape[0]
for i in range(1,num_rows): #перебор всех строк, начиная с 1
  if pd.isnull(summary_df.iloc[i]['Ф.И.О. пациента']) and ~pd.isnull(summary_df.iloc[i]['Ф.И.О. пациента']):
    #print ('заполнил фио',i)
    summary_df.iloc[i]['Ф.И.О. пациента']=summary_df.iloc[i-1]['Ф.И.О. пациента']#Если не заполнено поле с ФИО, берем предыдущий
  if summary_df.iloc[i]['Ф.И.О. пациента']!=summary_df.iloc[i-1]['Ф.И.О. пациента']:
    continue
  for column_index in columns_index: #перебор всех заполняемых столбцов
    if pd.isnull(summary_df.iloc[i,column_index]):
      summary_df.iloc[i,column_index]=summary_df.iloc[i-1,column_index]

In [21]:
summary_df

,МО прикрепления,Ф.И.О. пациента,Пол,Дата рождения,Диагноз,Сердечно -сосудистые события,Наименование препарата,Доза,Дата последнего анализа,Последнее значение
0,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-26 00:00:00,2.4
1,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-13 00:00:00,2.33
2,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-12-05 00:00:00,1.98
3,ГБУЗ РБ ГКБ №5,Егорова Антонида Александровна,женский,1952-02-06 00:00:00,Фибрилляция предсердий,NaN,Варфарин,1 табл.,2023-02-01 00:00:00,1.83
4,ГБУЗ РБ ГКБ №5,Егорова Антонида Александровна,женский,1952-02-06 00:00:00,Фибрилляция предсердий,NaN,Варфарин,1 табл.,2022-12-20 00:00:00,2.53
...,...,...,...,...,...,...,...,...,...,...
94,ГБУЗ РБ Буздякская ЦРБ,Юлдашева Фания Ризвановна,жен,1955-08-21 00:00:00,ИБС с нарушением ритма.Постоянная фибрилляция ...,NaN,Варфарин,"2,5 таб",2022-12-12 00:00:00,1.66
95,ГБУЗ РБ Буздякская ЦРБ,Юлдашева Фания Ризвановна,жен,1955-08-21 00:00:00,ИБС с нарушением ритма.Постоянная фибрилляция ...,NaN,Варфарин,"2,5 таб",2022-08-12 00:00:00,1.28
96,ГБУЗ РБ Буздякская ЦРБ,Амирова Елена Хатмиевна,жен,1952-04-25 00:00:00,ИБС.Персиситирующая форма фибрилляции предсердий,NaN,Варфарин,"1,75 таб",2022-12-29 00:00:00,2.99
97,ГБУЗ РБ Буздякская ЦРБ,Амирова Елена Хатмиевна,жен,1952-04-25 00:00:00,ИБС.Персиситирующая форма фибрилляции предсердий,NaN,Варфарин,"1,75 таб",2022-05-23 00:00:00,1.13


##Фильтрация по препарату

In [22]:
#Посмотрим варианты заполнения графы Наименование препарата
summary_df['Наименование препарата'].unique()

array(['Варфарин', 'варфарин', 'Варфарин\xa0 ', nan, 'Прадакса 150',
       'Прадакса150', 'Апиксабан 5', 'прадакса 150', 'Эликвис 5',
       'Ксарелта 20', ' Варфарин ', 'варфарин 2,5 мг', 'варфарин 2.5 мг',
       ' варфарин 2,5 мг ', 'варфарин2,5мг', 'варфарин 2.5мг',
       'варфарин  2.5 мг', 'Варфарин 2.5мг', 'Варфарин 2,5мг',
       'Варфарин 2.5 мг', 'Варфарин 2,5 мг', 'Варфарин 2,5 мг ',
       'варфарин ', 'ксарелто', 'Ксарелто', 'Варфарин ',
       '\xa0ривароксабан\xa0', 'Дабигатрана этексилат', 'Апиксабан',
       'Ривароксабан', 'ривароксабан ', 'равароксабан', 'ривароксабан',
       'ксарелто 20мг', 'эликвис 5мг ', 'Дабигатран 150 мг ',
       'эликвис 5 мг', 'Дабигатран 150 мг', 'Продакса 150 мг',
       'Апиксабан 5мг', 'НПОАК', 'эликвис', 'дабигатран', 'апиксабан'],
      dtype=object)

Очевидно, что есть много вариантов написания препарата. Создадим отдельный столбец, котоорый будет указывать варфарин или нет

In [23]:
summary_df['is_varfarin']= summary_df['Наименование препарата'].apply( lambda x: str(x).upper().find('ВАРФ')>=0)

In [24]:
summary_df=summary_df[summary_df['is_varfarin']]

In [25]:
summary_df

,МО прикрепления,Ф.И.О. пациента,Пол,Дата рождения,Диагноз,Сердечно -сосудистые события,Наименование препарата,Доза,Дата последнего анализа,Последнее значение,is_varfarin
0,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-26 00:00:00,2.4,True
1,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-13 00:00:00,2.33,True
2,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-12-05 00:00:00,1.98,True
3,ГБУЗ РБ ГКБ №5,Егорова Антонида Александровна,женский,1952-02-06 00:00:00,Фибрилляция предсердий,NaN,Варфарин,1 табл.,2023-02-01 00:00:00,1.83,True
4,ГБУЗ РБ ГКБ №5,Егорова Антонида Александровна,женский,1952-02-06 00:00:00,Фибрилляция предсердий,NaN,Варфарин,1 табл.,2022-12-20 00:00:00,2.53,True
...,...,...,...,...,...,...,...,...,...,...,...
94,ГБУЗ РБ Буздякская ЦРБ,Юлдашева Фания Ризвановна,жен,1955-08-21 00:00:00,ИБС с нарушением ритма.Постоянная фибрилляция ...,NaN,Варфарин,"2,5 таб",2022-12-12 00:00:00,1.66,True
95,ГБУЗ РБ Буздякская ЦРБ,Юлдашева Фания Ризвановна,жен,1955-08-21 00:00:00,ИБС с нарушением ритма.Постоянная фибрилляция ...,NaN,Варфарин,"2,5 таб",2022-08-12 00:00:00,1.28,True
96,ГБУЗ РБ Буздякская ЦРБ,Амирова Елена Хатмиевна,жен,1952-04-25 00:00:00,ИБС.Персиситирующая форма фибрилляции предсердий,NaN,Варфарин,"1,75 таб",2022-12-29 00:00:00,2.99,True
97,ГБУЗ РБ Буздякская ЦРБ,Амирова Елена Хатмиевна,жен,1952-04-25 00:00:00,ИБС.Персиситирующая форма фибрилляции предсердий,NaN,Варфарин,"1,75 таб",2022-05-23 00:00:00,1.13,True


In [26]:
#Еще раз Посмотрим варианты заполнения графы Наименование препарата
summary_df['Наименование препарата'].unique()

array(['Варфарин', 'варфарин', 'Варфарин\xa0 ', ' Варфарин ',
       'варфарин 2,5 мг', 'варфарин 2.5 мг', ' варфарин 2,5 мг ',
       'варфарин2,5мг', 'варфарин 2.5мг', 'варфарин  2.5 мг',
       'Варфарин 2.5мг', 'Варфарин 2,5мг', 'Варфарин 2.5 мг',
       'Варфарин 2,5 мг', 'Варфарин 2,5 мг ', 'варфарин ', 'Варфарин '],
      dtype=object)

##Формируем возраст

In [27]:
#Преобразоваваем дату рождения
summary_df['age']= summary_df.apply( lambda x: get_age_for_row (x),axis=1)
summary_df

Ошибка преобразования даты рождения: NAN
Ошибка преобразования даты рождения: NAN
Ошибка преобразования даты анализа: Ё
Ошибка преобразования даты анализа: 00:00:00
Ошибка преобразования даты рождения: NAN
Ошибка преобразования даты рождения: NAN
Ошибка преобразования даты рождения: NAN
Ошибка преобразования даты рождения: NAN
Ошибка преобразования даты рождения: NAN
Ошибка преобразования даты рождения: NAN
Ошибка преобразования даты анализа: 24.11.2022 Г ., 16.01.2023Г
Ошибка преобразования даты рождения: NAN
Ошибка преобразования даты рождения: NAN
Ошибка преобразования даты рождения: NAN


<ipython-input-27-4ce8e0ad9ec5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_df['age']= summary_df.apply( lambda x: get_age_for_row (x),axis=1)


,МО прикрепления,Ф.И.О. пациента,Пол,Дата рождения,Диагноз,Сердечно -сосудистые события,Наименование препарата,Доза,Дата последнего анализа,Последнее значение,is_varfarin,age
0,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-26 00:00:00,2.4,True,69
1,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-13 00:00:00,2.33,True,69
2,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-12-05 00:00:00,1.98,True,70
3,ГБУЗ РБ ГКБ №5,Егорова Антонида Александровна,женский,1952-02-06 00:00:00,Фибрилляция предсердий,NaN,Варфарин,1 табл.,2023-02-01 00:00:00,1.83,True,71
4,ГБУЗ РБ ГКБ №5,Егорова Антонида Александровна,женский,1952-02-06 00:00:00,Фибрилляция предсердий,NaN,Варфарин,1 табл.,2022-12-20 00:00:00,2.53,True,70
...,...,...,...,...,...,...,...,...,...,...,...,...
94,ГБУЗ РБ Буздякская ЦРБ,Юлдашева Фания Ризвановна,жен,1955-08-21 00:00:00,ИБС с нарушением ритма.Постоянная фибрилляция ...,NaN,Варфарин,"2,5 таб",2022-12-12 00:00:00,1.66,True,67
95,ГБУЗ РБ Буздякская ЦРБ,Юлдашева Фания Ризвановна,жен,1955-08-21 00:00:00,ИБС с нарушением ритма.Постоянная фибрилляция ...,NaN,Варфарин,"2,5 таб",2022-08-12 00:00:00,1.28,True,67
96,ГБУЗ РБ Буздякская ЦРБ,Амирова Елена Хатмиевна,жен,1952-04-25 00:00:00,ИБС.Персиситирующая форма фибрилляции предсердий,NaN,Варфарин,"1,75 таб",2022-12-29 00:00:00,2.99,True,70
97,ГБУЗ РБ Буздякская ЦРБ,Амирова Елена Хатмиевна,жен,1952-04-25 00:00:00,ИБС.Персиситирующая форма фибрилляции предсердий,NaN,Варфарин,"1,75 таб",2022-05-23 00:00:00,1.13,True,70


In [28]:
#Фамилии пациетов, для которых ошибка определения возраста
PatientList=list(summary_df[summary_df['age']==0]['Ф.И.О. пациента'].unique())
PatientList

['Халиуллина Светлана Геннадьевна',
 'Булякова Миннур Галимьяновна',
 'Дмитриева Галина Николаевна',
 'Лобачева Надежда Романовна',
 'ЗИННАТУЛЛИН Марс Гиндуллинович ',
 'Фатыхова Гульсина Шакировна',
 'Валеева Альфия Гафуровна ',
 'Губайдуллина Бибикамал Абдрахмановна',
 'Бондарева Надежда Алексеевна']

In [29]:
summary_df[summary_df['Ф.И.О. пациента'].isin(PatientList)]

,МО прикрепления,Ф.И.О. пациента,Пол,Дата рождения,Диагноз,Сердечно -сосудистые события,Наименование препарата,Доза,Дата последнего анализа,Последнее значение,is_varfarin,age
55,ЧУЗ КБ РЖД-Медицина г.Уфа,Халиуллина Светлана Геннадьевна,женский,NaN,Протезирование аортального клапана,NaN,Варфарин,2 табл.,2022-10-24 00:00:00,1.95,True,0
56,ЧУЗ КБ РЖД-Медицина г.Уфа,Халиуллина Светлана Геннадьевна,женский,NaN,Протезирование аортального клапана,NaN,Варфарин,2 табл.,2022-09-06 00:00:00,2.15,True,0
268,ЦАТ_SIB,Булякова Миннур Галимьяновна,ж,2022-06-17 00:00:00,ИБС.стенок,нет,Варфарин,1т,2022-11-09 00:00:00,0.94,True,0
220,Белебеевская ЦРБ,Дмитриева Галина Николаевна,ж,1956-03-17 00:00:00,"протезирование митрального клапана,фибрилляция...","Стентирование ПМЖВ 01.06.18г, стентирование П...",варфарин 2.5 мг,2 т,ё,1.25,True,0
221,Белебеевская ЦРБ,Дмитриева Галина Николаевна,ж,1956-03-17 00:00:00,"протезирование митрального клапана,фибрилляция...","Стентирование ПМЖВ 01.06.18г, стентирование П...",варфарин 2.5 мг,2 т,2023-01-16 00:00:00,6.8,True,66
222,Белебеевская ЦРБ,Дмитриева Галина Николаевна,ж,1956-03-17 00:00:00,"протезирование митрального клапана,фибрилляция...","Стентирование ПМЖВ 01.06.18г, стентирование П...",варфарин 2.5 мг,2 т,2023-01-23 00:00:00,3.9,True,66
700,г.Октябрьский поликлиника №1,Лобачева Надежда Романовна,ж,1965-07-01 00:00:00,"Протезирование МК, пластика ТК 09.02.2009г",NaN,"Варфарин 2,5мг",1т,00:00:00,2.94,True,0
701,г.Октябрьский поликлиника №1,Лобачева Надежда Романовна,ж,1965-07-01 00:00:00,"Протезирование МК, пластика ТК 09.02.2009г",NaN,"Варфарин 2,5мг",1т,2022-11-15 00:00:00,2.6,True,57
702,г.Октябрьский поликлиника №1,Лобачева Надежда Романовна,ж,1965-07-01 00:00:00,"Протезирование МК, пластика ТК 09.02.2009г",NaN,"Варфарин 2,5мг",1т,2022-11-25 00:00:00,2.94,True,57
717,ГБУЗ РБ Верхнеяркеевская ЦРБ,ЗИННАТУЛЛИН Марс Гиндуллинович,м,NaN,Протезирование аортального клапана\n фибрилля...,NaN,варфарин,3 таб,2022-08-01 00:00:00,2.3,True,0


Очевидно, для данных пациентов не хватает данных, можно смело удалить эти записи

In [30]:
#Удаляем строки, для которых age==0
summary_df=summary_df[summary_df['age']>0]
summary_df

,МО прикрепления,Ф.И.О. пациента,Пол,Дата рождения,Диагноз,Сердечно -сосудистые события,Наименование препарата,Доза,Дата последнего анализа,Последнее значение,is_varfarin,age
0,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-26 00:00:00,2.4,True,69
1,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-13 00:00:00,2.33,True,69
2,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-12-05 00:00:00,1.98,True,70
3,ГБУЗ РБ ГКБ №5,Егорова Антонида Александровна,женский,1952-02-06 00:00:00,Фибрилляция предсердий,NaN,Варфарин,1 табл.,2023-02-01 00:00:00,1.83,True,71
4,ГБУЗ РБ ГКБ №5,Егорова Антонида Александровна,женский,1952-02-06 00:00:00,Фибрилляция предсердий,NaN,Варфарин,1 табл.,2022-12-20 00:00:00,2.53,True,70
...,...,...,...,...,...,...,...,...,...,...,...,...
94,ГБУЗ РБ Буздякская ЦРБ,Юлдашева Фания Ризвановна,жен,1955-08-21 00:00:00,ИБС с нарушением ритма.Постоянная фибрилляция ...,NaN,Варфарин,"2,5 таб",2022-12-12 00:00:00,1.66,True,67
95,ГБУЗ РБ Буздякская ЦРБ,Юлдашева Фания Ризвановна,жен,1955-08-21 00:00:00,ИБС с нарушением ритма.Постоянная фибрилляция ...,NaN,Варфарин,"2,5 таб",2022-08-12 00:00:00,1.28,True,67
96,ГБУЗ РБ Буздякская ЦРБ,Амирова Елена Хатмиевна,жен,1952-04-25 00:00:00,ИБС.Персиситирующая форма фибрилляции предсердий,NaN,Варфарин,"1,75 таб",2022-12-29 00:00:00,2.99,True,70
97,ГБУЗ РБ Буздякская ЦРБ,Амирова Елена Хатмиевна,жен,1952-04-25 00:00:00,ИБС.Персиситирующая форма фибрилляции предсердий,NaN,Варфарин,"1,75 таб",2022-05-23 00:00:00,1.13,True,70


##Приведем к единому виду столбец с датой анализа

In [31]:
summary_df['analiz_date']= summary_df['Дата последнего анализа'].apply( lambda x: get_valid_date (str(x)))
summary_df

<ipython-input-31-87d9d8f6624a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_df['analiz_date']= summary_df['Дата последнего анализа'].apply( lambda x: get_valid_date (str(x)))


,МО прикрепления,Ф.И.О. пациента,Пол,Дата рождения,Диагноз,Сердечно -сосудистые события,Наименование препарата,Доза,Дата последнего анализа,Последнее значение,is_varfarin,age,analiz_date
0,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-26 00:00:00,2.4,True,69,2023-01-26
1,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-13 00:00:00,2.33,True,69,2023-01-13
2,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-12-05 00:00:00,1.98,True,70,2023-12-05
3,ГБУЗ РБ ГКБ №5,Егорова Антонида Александровна,женский,1952-02-06 00:00:00,Фибрилляция предсердий,NaN,Варфарин,1 табл.,2023-02-01 00:00:00,1.83,True,71,2023-02-01
4,ГБУЗ РБ ГКБ №5,Егорова Антонида Александровна,женский,1952-02-06 00:00:00,Фибрилляция предсердий,NaN,Варфарин,1 табл.,2022-12-20 00:00:00,2.53,True,70,2022-12-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,ГБУЗ РБ Буздякская ЦРБ,Юлдашева Фания Ризвановна,жен,1955-08-21 00:00:00,ИБС с нарушением ритма.Постоянная фибрилляция ...,NaN,Варфарин,"2,5 таб",2022-12-12 00:00:00,1.66,True,67,2022-12-12
95,ГБУЗ РБ Буздякская ЦРБ,Юлдашева Фания Ризвановна,жен,1955-08-21 00:00:00,ИБС с нарушением ритма.Постоянная фибрилляция ...,NaN,Варфарин,"2,5 таб",2022-08-12 00:00:00,1.28,True,67,2022-08-12
96,ГБУЗ РБ Буздякская ЦРБ,Амирова Елена Хатмиевна,жен,1952-04-25 00:00:00,ИБС.Персиситирующая форма фибрилляции предсердий,NaN,Варфарин,"1,75 таб",2022-12-29 00:00:00,2.99,True,70,2022-12-29
97,ГБУЗ РБ Буздякская ЦРБ,Амирова Елена Хатмиевна,жен,1952-04-25 00:00:00,ИБС.Персиситирующая форма фибрилляции предсердий,NaN,Варфарин,"1,75 таб",2022-05-23 00:00:00,1.13,True,70,2022-05-23


##Преобразуем стобец "Пол"

In [32]:
def get_gender(arg:str):
  arg=arg.strip().upper()
  if arg.startswith('M') or arg.startswith('М'):
    return 1
  return 0

In [33]:
summary_df['gender']= summary_df['Пол'].apply( lambda x: get_gender (str(x)))
summary_df

<ipython-input-33-213635559a57>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_df['gender']= summary_df['Пол'].apply( lambda x: get_gender (str(x)))


,МО прикрепления,Ф.И.О. пациента,Пол,Дата рождения,Диагноз,Сердечно -сосудистые события,Наименование препарата,Доза,Дата последнего анализа,Последнее значение,is_varfarin,age,analiz_date,gender
0,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-26 00:00:00,2.4,True,69,2023-01-26,0
1,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-13 00:00:00,2.33,True,69,2023-01-13,0
2,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-12-05 00:00:00,1.98,True,70,2023-12-05,0
3,ГБУЗ РБ ГКБ №5,Егорова Антонида Александровна,женский,1952-02-06 00:00:00,Фибрилляция предсердий,NaN,Варфарин,1 табл.,2023-02-01 00:00:00,1.83,True,71,2023-02-01,0
4,ГБУЗ РБ ГКБ №5,Егорова Антонида Александровна,женский,1952-02-06 00:00:00,Фибрилляция предсердий,NaN,Варфарин,1 табл.,2022-12-20 00:00:00,2.53,True,70,2022-12-20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,ГБУЗ РБ Буздякская ЦРБ,Юлдашева Фания Ризвановна,жен,1955-08-21 00:00:00,ИБС с нарушением ритма.Постоянная фибрилляция ...,NaN,Варфарин,"2,5 таб",2022-12-12 00:00:00,1.66,True,67,2022-12-12,0
95,ГБУЗ РБ Буздякская ЦРБ,Юлдашева Фания Ризвановна,жен,1955-08-21 00:00:00,ИБС с нарушением ритма.Постоянная фибрилляция ...,NaN,Варфарин,"2,5 таб",2022-08-12 00:00:00,1.28,True,67,2022-08-12,0
96,ГБУЗ РБ Буздякская ЦРБ,Амирова Елена Хатмиевна,жен,1952-04-25 00:00:00,ИБС.Персиситирующая форма фибрилляции предсердий,NaN,Варфарин,"1,75 таб",2022-12-29 00:00:00,2.99,True,70,2022-12-29,0
97,ГБУЗ РБ Буздякская ЦРБ,Амирова Елена Хатмиевна,жен,1952-04-25 00:00:00,ИБС.Персиситирующая форма фибрилляции предсердий,NaN,Варфарин,"1,75 таб",2022-05-23 00:00:00,1.13,True,70,2022-05-23,0


#Дозировка

In [34]:
#Посмотрим варианты заполнения графы Доза
DozaList=list(summary_df['Доза'].unique())
print(f'Всего вариантов: {len(DozaList)}')
DozaList

Всего вариантов: 384


['2,5 табл',
 '1 табл.',
 '2 табл.',
 '3.0',
 2.5,
 5.5,
 5.625,
 3.25,
 3.125,
 3,
 5.75,
 7.5,
 '2,5мг по  1+3/4 таб вечером',
 '2,5мг по 2+1/4 таблетки вечером',
 '2,5 мг по 2+1/4 таблетки ечером',
 '2,5мг по  1+1/4 таблетке вечером',
 '       ',
 '2,5мг по 1,5 таблетки 5 дней, затем 3/4 2 дней',
 '2,5мг по 1/2 таблетки 1 раз в день',
 '2,5мг по 3/4 таблетки 1 раз в день',
 ' 2,5 мг по1 + 3/4 таблетки 1 раз в день',
 5,
 2.625,
 4.375,
 3.75,
 '1.75 табл',
 '2 табл',
 '1,5 табл',
 '1,75 табл',
 '1,25 таб',
 '2 таб',
 '1,5 таб',
 '0,5 таб',
 '1,75 таб',
 '2,5 мг 2,5 таб',
 ' 2,5 таб',
 ' 1 таб',
 '2таб',
 '1,75таб',
 '2,5-2,75 таб через день',
 '4 таб',
 '1,25 - 1,5 таб через день',
 '0,75 таб',
 '1 таб',
 '1таб',
 '2,5 мг 1 таб',
 '2,5 мг 1,25 таб',
 ' 1,25 таб',
 '1 ТАБ',
 '1 таб ',
 '1,5 таб ',
 '2,25 таб ',
 '2,25 таб',
 '2,5 мг 1,5 таб',
 ' 1,5 таб',
 2.75,
 '2,75 таб',
 '2,5 мг 3 таб',
 ' 3 таб',
 '1,5  таб',
 '2,5 мг 0,5 таб',
 '2,5 мг 2 таб',
 '2 ТАБ',
 '2 таб ',
 ' 1т+1/4т',

In [35]:
def str2float(arg:str):
  arg=arg.strip()
  arg=arg.replace(',','.')
  #Пытаемся преобразовать строку сразу в число
  res=0
  try:
    res=float(arg)
  except ValueError as err:
    return None
  return res

In [36]:
TABDOZE=2500
def get_doza(arg:str):
  '''
  получает строку Доза возвращает дозировку в мг
  '''
  arg=arg.strip()
  arg=arg.replace(',','.')
  #Пытаемся преобразовать строку сразу в число
  res=str2float(arg)
  if res is not None:
    if res<10:
      return res
    return res*TABDOZE
  #Пробуем с помощью регулярного  выражения узнать количество таблеток
  tab_arg=arg.replace('\\','/').replace('1/2','0.5').replace('1/4','0.25').replace('3/4','0.75')
  pattern=r'([\d.]{1,4})[\s]{0,5}[тТ]{1}'
  result = re.findall(pattern, arg)
  if len(result)==1:
    tab_count= str2float(result[0])
    return tab_count*TABDOZE



  return None

In [37]:
arg='5 таб 2 таб'
print (get_doza(arg))

None


In [38]:
summary_df['Doza']=summary_df['Доза'].apply(lambda x: get_doza(str(x)))
summary_df

<ipython-input-38-ab53cb6c4276>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_df['Doza']=summary_df['Доза'].apply(lambda x: get_doza(str(x)))


,МО прикрепления,Ф.И.О. пациента,Пол,Дата рождения,Диагноз,Сердечно -сосудистые события,Наименование препарата,Доза,Дата последнего анализа,Последнее значение,is_varfarin,age,analiz_date,gender,Doza
0,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-26 00:00:00,2.4,True,69,2023-01-26,0,6250.0
1,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-13 00:00:00,2.33,True,69,2023-01-13,0,6250.0
2,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-12-05 00:00:00,1.98,True,70,2023-12-05,0,6250.0
3,ГБУЗ РБ ГКБ №5,Егорова Антонида Александровна,женский,1952-02-06 00:00:00,Фибрилляция предсердий,NaN,Варфарин,1 табл.,2023-02-01 00:00:00,1.83,True,71,2023-02-01,0,2500.0
4,ГБУЗ РБ ГКБ №5,Егорова Антонида Александровна,женский,1952-02-06 00:00:00,Фибрилляция предсердий,NaN,Варфарин,1 табл.,2022-12-20 00:00:00,2.53,True,70,2022-12-20,0,2500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,ГБУЗ РБ Буздякская ЦРБ,Юлдашева Фания Ризвановна,жен,1955-08-21 00:00:00,ИБС с нарушением ритма.Постоянная фибрилляция ...,NaN,Варфарин,"2,5 таб",2022-12-12 00:00:00,1.66,True,67,2022-12-12,0,6250.0
95,ГБУЗ РБ Буздякская ЦРБ,Юлдашева Фания Ризвановна,жен,1955-08-21 00:00:00,ИБС с нарушением ритма.Постоянная фибрилляция ...,NaN,Варфарин,"2,5 таб",2022-08-12 00:00:00,1.28,True,67,2022-08-12,0,6250.0
96,ГБУЗ РБ Буздякская ЦРБ,Амирова Елена Хатмиевна,жен,1952-04-25 00:00:00,ИБС.Персиситирующая форма фибрилляции предсердий,NaN,Варфарин,"1,75 таб",2022-12-29 00:00:00,2.99,True,70,2022-12-29,0,4375.0
97,ГБУЗ РБ Буздякская ЦРБ,Амирова Елена Хатмиевна,жен,1952-04-25 00:00:00,ИБС.Персиситирующая форма фибрилляции предсердий,NaN,Варфарин,"1,75 таб",2022-05-23 00:00:00,1.13,True,70,2022-05-23,0,4375.0


##Диагнозы


Для определения диагнозов воспользуемся датасетом Сергея Кима, в котором эта работа проделана вручную

In [41]:
#грузим файлы Сергея Кима
#ссылка на моем гугл диске: https://drive.google.com/file/d/1otJ3TuGuJ78SCTFjGKm958m12g8Ho6Xr/view?usp=sharing
KimDataSetURL = "https://drive.google.com/uc?export=download&id=1otJ3TuGuJ78SCTFjGKm958m12g8Ho6Xr"

r = requests.get(KimDataSetURL)
with open("Датасет Сергея Кима.zip",'wb') as f:
  f.write(r.content)
!unzip /content/"Датасет Сергея Кима.zip" -d $DatasetDir

Archive:  /content/Датасет Сергея Кима.zip
  inflating: /content/dataset/Датасет Сергея Кима.xlsx  


In [44]:
#формируем общий датафрейм
Kim_df=pd.read_excel("/content/dataset/Датасет Сергея Кима.xlsx",sheet_name="dataForAll")

In [45]:
Kim_df

,Name,patient,gender,age,diagnos,sss,mg,AnalisDate,MNO,ИндексДиаг,ИндексПац,ИндексSSS
0,ЦАТ_SIBа.xlsx,Абдрахимов Гайфулла Рахимьянович,0,64.317591,ИБС,0,2.500,2022-11-28,2.46,3,484,1
1,ЦАТ_KMRTа.xlsx,Абдрахманов Гаряй Анварович,0,65.382615,ИБС ПС,ПК,7.500,2022-09-16,3.41,8,135,3
2,ЦАТ_KMRTа.xlsx,Абдрахманов Гаряй Анварович,0,65.382615,ПС,ПК,8.125,2022-11-02,1.89,2,135,3
3,ЦАТ_KMRTа.xlsx,Абдрахманов Гаряй Анварович,0,65.382615,ПС,ПК,8.125,2022-12-28,1.59,2,135,3
4,ЦАТ_KMRTа.xlsx,Абдрахманова Минигуль Габдуловна,1,69.911020,ПС,ПК,8.125,2022-03-23,2.87,2,136,3
...,...,...,...,...,...,...,...,...,...,...,...,...
1912,ЦАТ_GKB5а.xlsx,Яхин Махмут Шарифуллович,0,76.479124,ФП,0,2.500,2022-12-21,2.43,1,73,1
1913,ЦАТ_GKB5а.xlsx,Яхин Махмут Шарифуллович,0,76.479124,ФП,0,2.500,2023-12-19,2.16,1,73,1
1914,ЦАТ_SIBа.xlsx,Яценко Людмила Афанасьевна,1,81.900068,ФП,0,2.500,2022-12-28,1.14,1,485,1
1915,ЦАТ_SIBа.xlsx,Яценко Людмила Афанасьевна,1,81.900068,ФП,0,2.500,2023-01-24,2.87,1,485,1


In [112]:
result_df=summary_df.merge(Kim_df[['patient','ИндексДиаг','ИндексSSS']],left_on="Ф.И.О. пациента",right_on="patient")
result_df

,МО прикрепления,Ф.И.О. пациента,Пол,Дата рождения,Диагноз,Сердечно -сосудистые события,Наименование препарата,Доза,Дата последнего анализа,Последнее значение,is_varfarin,age,analiz_date,gender,Doza,patient,ИндексДиаг,ИндексSSS
0,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-26 00:00:00,2.4,True,69,2023-01-26,0,6250.0,Шаяхметова Рузалия Валетдиновна,8,1
1,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-26 00:00:00,2.4,True,69,2023-01-26,0,6250.0,Шаяхметова Рузалия Валетдиновна,8,1
2,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-26 00:00:00,2.4,True,69,2023-01-26,0,6250.0,Шаяхметова Рузалия Валетдиновна,8,1
3,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-13 00:00:00,2.33,True,69,2023-01-13,0,6250.0,Шаяхметова Рузалия Валетдиновна,8,1
4,ГБУЗ РБ ГКБ №5,Шаяхметова Рузалия Валетдиновна,женский,1953-06-25 00:00:00,Протезирование аортального клапана,ОНМК 2021,Варфарин,"2,5 табл",2023-01-13 00:00:00,2.33,True,69,2023-01-13,0,6250.0,Шаяхметова Рузалия Валетдиновна,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4408,ГБУЗ РБ Буздякская ЦРБ,Юлдашева Фания Ризвановна,жен,1955-08-21 00:00:00,ИБС с нарушением ритма.Постоянная фибрилляция ...,NaN,Варфарин,"2,5 таб",2022-12-12 00:00:00,1.66,True,67,2022-12-12,0,6250.0,Юлдашева Фания Ризвановна,3,3
4409,ГБУЗ РБ Буздякская ЦРБ,Юлдашева Фания Ризвановна,жен,1955-08-21 00:00:00,ИБС с нарушением ритма.Постоянная фибрилляция ...,NaN,Варфарин,"2,5 таб",2022-12-12 00:00:00,1.66,True,67,2022-12-12,0,6250.0,Юлдашева Фания Ризвановна,3,3
4410,ГБУЗ РБ Буздякская ЦРБ,Юлдашева Фания Ризвановна,жен,1955-08-21 00:00:00,ИБС с нарушением ритма.Постоянная фибрилляция ...,NaN,Варфарин,"2,5 таб",2022-08-12 00:00:00,1.28,True,67,2022-08-12,0,6250.0,Юлдашева Фания Ризвановна,3,3
4411,ГБУЗ РБ Буздякская ЦРБ,Юлдашева Фания Ризвановна,жен,1955-08-21 00:00:00,ИБС с нарушением ритма.Постоянная фибрилляция ...,NaN,Варфарин,"2,5 таб",2022-08-12 00:00:00,1.28,True,67,2022-08-12,0,6250.0,Юлдашева Фания Ризвановна,3,3


In [113]:
#Теперь можно удалить лишние столбцы
result_df.drop(['МО прикрепления','Пол','Дата рождения ','Диагноз','Сердечно -сосудистые события','Наименование препарата','is_varfarin','Доза','patient'],axis=1,inplace=True)

In [114]:
result_df

,Ф.И.О. пациента,Дата последнего анализа,Последнее значение,age,analiz_date,gender,Doza,ИндексДиаг,ИндексSSS
0,Шаяхметова Рузалия Валетдиновна,2023-01-26 00:00:00,2.4,69,2023-01-26,0,6250.0,8,1
1,Шаяхметова Рузалия Валетдиновна,2023-01-26 00:00:00,2.4,69,2023-01-26,0,6250.0,8,1
2,Шаяхметова Рузалия Валетдиновна,2023-01-26 00:00:00,2.4,69,2023-01-26,0,6250.0,8,1
3,Шаяхметова Рузалия Валетдиновна,2023-01-13 00:00:00,2.33,69,2023-01-13,0,6250.0,8,1
4,Шаяхметова Рузалия Валетдиновна,2023-01-13 00:00:00,2.33,69,2023-01-13,0,6250.0,8,1
...,...,...,...,...,...,...,...,...,...
4408,Юлдашева Фания Ризвановна,2022-12-12 00:00:00,1.66,67,2022-12-12,0,6250.0,3,3
4409,Юлдашева Фания Ризвановна,2022-12-12 00:00:00,1.66,67,2022-12-12,0,6250.0,3,3
4410,Юлдашева Фания Ризвановна,2022-08-12 00:00:00,1.28,67,2022-08-12,0,6250.0,3,3
4411,Юлдашева Фания Ризвановна,2022-08-12 00:00:00,1.28,67,2022-08-12,0,6250.0,3,3


In [115]:
#сортируем датасет по колонкам 'Ф.И.О. пациента' и analiz_date
result_df=result_df.sort_values(['Ф.И.О. пациента','analiz_date'])

In [116]:
result_df

,Ф.И.О. пациента,Дата последнего анализа,Последнее значение,age,analiz_date,gender,Doza,ИндексДиаг,ИндексSSS
4048,АБРАМОВ ТАТЬЯНА,2022-12-11 00:00:00,"1 ,5",75,2022-12-11,0,2500.0,1,1
4049,АБРАМОВ ТАТЬЯНА,2022-12-11 00:00:00,"1 ,5",75,2022-12-11,0,2500.0,1,1
4050,АБРАМОВ ТАТЬЯНА,2023-01-24 00:00:00,1.8,75,2023-01-24,0,2500.0,1,1
4051,АБРАМОВ ТАТЬЯНА,2023-01-24 00:00:00,1.8,75,2023-01-24,0,2500.0,1,1
1414,Абдрахимов Гайфулла Рахимьянович,2022-11-28 00:00:00,2.46,64,2022-11-28,1,2500.0,3,1
...,...,...,...,...,...,...,...,...,...
3839,мирзаянов Альфред Амирьянович,2023-01-13 00:00:00,3,66,2023-01-13,1,NaN,2,1
3840,мирзаянов Альфред Амирьянович,2023-01-13 00:00:00,3,66,2023-01-13,1,NaN,2,1
3835,мирзаянов Альфред Амирьянович,2023-01-23 00:00:00,3.74,66,2023-01-23,1,5000.0,2,1
3836,мирзаянов Альфред Амирьянович,2023-01-23 00:00:00,3.74,66,2023-01-23,1,5000.0,2,1


In [117]:
#удалим строки с Nan
result_df=result_df.dropna(how="any")
result_df

,Ф.И.О. пациента,Дата последнего анализа,Последнее значение,age,analiz_date,gender,Doza,ИндексДиаг,ИндексSSS
4048,АБРАМОВ ТАТЬЯНА,2022-12-11 00:00:00,"1 ,5",75,2022-12-11,0,2500.00,1,1
4049,АБРАМОВ ТАТЬЯНА,2022-12-11 00:00:00,"1 ,5",75,2022-12-11,0,2500.00,1,1
4050,АБРАМОВ ТАТЬЯНА,2023-01-24 00:00:00,1.8,75,2023-01-24,0,2500.00,1,1
4051,АБРАМОВ ТАТЬЯНА,2023-01-24 00:00:00,1.8,75,2023-01-24,0,2500.00,1,1
1414,Абдрахимов Гайфулла Рахимьянович,2022-11-28 00:00:00,2.46,64,2022-11-28,1,2500.00,3,1
...,...,...,...,...,...,...,...,...,...
2366,каюмов Флюз Вазирович,2023-01-20 00:00:00,2.5,55,2023-01-20,1,6.25,2,1
2367,каюмов Флюз Вазирович,2023-01-20 00:00:00,2.5,55,2023-01-20,1,6.25,2,1
3835,мирзаянов Альфред Амирьянович,2023-01-23 00:00:00,3.74,66,2023-01-23,1,5000.00,2,1
3836,мирзаянов Альфред Амирьянович,2023-01-23 00:00:00,3.74,66,2023-01-23,1,5000.00,2,1


##Добавим в таблицу столбец с результитрующим MHO

In [118]:
result_df.rename(columns={'Последнее значение':'Prev_MNO'},inplace=True)

<ipython-input-118-9d8417694ac6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.rename(columns={'Последнее значение':'Prev_MNO'},inplace=True)


In [119]:
result_df

,Ф.И.О. пациента,Дата последнего анализа,Prev_MNO,age,analiz_date,gender,Doza,ИндексДиаг,ИндексSSS
4048,АБРАМОВ ТАТЬЯНА,2022-12-11 00:00:00,"1 ,5",75,2022-12-11,0,2500.00,1,1
4049,АБРАМОВ ТАТЬЯНА,2022-12-11 00:00:00,"1 ,5",75,2022-12-11,0,2500.00,1,1
4050,АБРАМОВ ТАТЬЯНА,2023-01-24 00:00:00,1.8,75,2023-01-24,0,2500.00,1,1
4051,АБРАМОВ ТАТЬЯНА,2023-01-24 00:00:00,1.8,75,2023-01-24,0,2500.00,1,1
1414,Абдрахимов Гайфулла Рахимьянович,2022-11-28 00:00:00,2.46,64,2022-11-28,1,2500.00,3,1
...,...,...,...,...,...,...,...,...,...
2366,каюмов Флюз Вазирович,2023-01-20 00:00:00,2.5,55,2023-01-20,1,6.25,2,1
2367,каюмов Флюз Вазирович,2023-01-20 00:00:00,2.5,55,2023-01-20,1,6.25,2,1
3835,мирзаянов Альфред Амирьянович,2023-01-23 00:00:00,3.74,66,2023-01-23,1,5000.00,2,1
3836,мирзаянов Альфред Амирьянович,2023-01-23 00:00:00,3.74,66,2023-01-23,1,5000.00,2,1


In [120]:
result_df.insert(9,'MNO',np.zeros(result_df.shape[0])) #Добавление нового столбца, заполненого 0

In [121]:
for i in range(1,num_rows): #перебор всех строк, начиная с 1
  if result_df.iloc[i]['Ф.И.О. пациента']!=result_df.iloc[i-1]['Ф.И.О. пациента']:
    continue
  result_df.iloc[i-1,9]=str2float(str(result_df.iloc[i]['Prev_MNO']))

In [122]:
result_df

,Ф.И.О. пациента,Дата последнего анализа,Prev_MNO,age,analiz_date,gender,Doza,ИндексДиаг,ИндексSSS,MNO
4048,АБРАМОВ ТАТЬЯНА,2022-12-11 00:00:00,"1 ,5",75,2022-12-11,0,2500.00,1,1,NaN
4049,АБРАМОВ ТАТЬЯНА,2022-12-11 00:00:00,"1 ,5",75,2022-12-11,0,2500.00,1,1,1.8
4050,АБРАМОВ ТАТЬЯНА,2023-01-24 00:00:00,1.8,75,2023-01-24,0,2500.00,1,1,1.8
4051,АБРАМОВ ТАТЬЯНА,2023-01-24 00:00:00,1.8,75,2023-01-24,0,2500.00,1,1,0.0
1414,Абдрахимов Гайфулла Рахимьянович,2022-11-28 00:00:00,2.46,64,2022-11-28,1,2500.00,3,1,0.0
...,...,...,...,...,...,...,...,...,...,...
2366,каюмов Флюз Вазирович,2023-01-20 00:00:00,2.5,55,2023-01-20,1,6.25,2,1,0.0
2367,каюмов Флюз Вазирович,2023-01-20 00:00:00,2.5,55,2023-01-20,1,6.25,2,1,0.0
3835,мирзаянов Альфред Амирьянович,2023-01-23 00:00:00,3.74,66,2023-01-23,1,5000.00,2,1,0.0
3836,мирзаянов Альфред Амирьянович,2023-01-23 00:00:00,3.74,66,2023-01-23,1,5000.00,2,1,0.0


In [126]:
#удалим строки, в которых MNO=0
result_df=result_df[result_df['MNO']>0]

In [127]:
result_df

,Ф.И.О. пациента,Дата последнего анализа,Prev_MNO,age,analiz_date,gender,Doza,ИндексДиаг,ИндексSSS,MNO
4049,АБРАМОВ ТАТЬЯНА,2022-12-11 00:00:00,"1 ,5",75,2022-12-11,0,2500.0,1,1,1.80
4050,АБРАМОВ ТАТЬЯНА,2023-01-24 00:00:00,1.8,75,2023-01-24,0,2500.0,1,1,1.80
504,Абдулганиева Насима Мазгаровна,2022-09-28 00:00:00,1.9,64,2022-09-28,0,5000.0,2,1,1.90
505,Абдулганиева Насима Мазгаровна,2022-09-28 00:00:00,1.9,64,2022-09-28,0,5000.0,2,1,1.90
506,Абдулганиева Насима Мазгаровна,2022-09-28 00:00:00,1.9,64,2022-09-28,0,5000.0,2,1,2.90
...,...,...,...,...,...,...,...,...,...,...
2245,Тулбаева Глюза Узбековна,2022-11-25 00:00:00,1.23,62,2022-11-25,0,2500.0,1,1,1.23
2246,Тулбаева Глюза Узбековна,2022-11-25 00:00:00,1.23,62,2022-11-25,0,2500.0,1,1,1.23
2247,Тулбаева Глюза Узбековна,2022-11-25 00:00:00,1.23,62,2022-11-25,0,2500.0,1,1,2.10
2248,Тулбаева Глюза Узбековна,2022-12-12 00:00:00,2.1,62,2022-12-12,0,2500.0,1,1,2.10
